In [1]:
import pickle

Cargar Datos

In [6]:

with open('entrenamiento.p', 'rb') as file:
    training_data = pickle.load(file)
    

with open('prueba.p', 'rb') as file:
    test_data = pickle.load(file)
    

with open('validacion.p', 'rb') as file:
    validation_data = pickle.load(file)


In [7]:
X_train = training_data["features"]
Y_train = training_data["labels"]

In [8]:
X_test = test_data["features"]
y_test = test_data["labels"]

In [ ]:
X_val = validation_data["features"]
Y_val = test_data["labels"]

{'sizes': array([[53, 54],
        [42, 45],
        [48, 52],
        ...,
        [29, 29],
        [48, 49],
        [32, 31]], dtype=uint8),
 'coords': array([[ 6,  5, 48, 49],
        [ 5,  5, 36, 40],
        [ 6,  6, 43, 47],
        ...,
        [ 6,  6, 24, 24],
        [ 5,  6, 43, 44],
        [ 6,  5, 27, 26]], dtype=uint8),
 'features': array([[[[116, 139, 174],
          [116, 137, 171],
          [118, 138, 172],
          ...,
          [ 98, 114, 143],
          [ 97, 121, 147],
          [ 85, 105, 130]],
 
         [[115, 142, 176],
          [115, 140, 174],
          [117, 141, 175],
          ...,
          [119, 143, 176],
          [120, 141, 173],
          [120, 139, 171]],
 
         [[117, 141, 174],
          [117, 142, 175],
          [114, 140, 172],
          ...,
          [120, 144, 179],
          [122, 144, 179],
          [120, 143, 177]],
 
         ...,
 
         [[117, 137, 167],
          [115, 134, 164],
          [117, 135, 164],
          ..